 mlflow ui --backend-store-uri sqlite:///mlruns.db

In [17]:
#Importações
import pandas as pd
from sklearn.model_selection import train_test_split
import mlflow
from pycaret.datasets import get_data
from pycaret.classification import *
import os
from io import StringIO


In [18]:
# importação data set  https://www.kaggle.com/c/kobe-bryant-shot-selection/data
df = pd.read_parquet("../data/raw/dataset_kobe_dev.parquet")
df.shape
# df_origianal.columns

#df_prod = pd.read_parquet("../data/raw/dataset_kobe_prod.parquet")

(24271, 25)

In [19]:
# Para usar o sqlite como repositorio
mlflow.set_tracking_uri("sqlite:///mlruns.db")

experiment_name = 'Aula Modelos'
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment(experiment_id)
experiment_id = experiment.experiment_id

In [20]:
def adicionaDataFrameArtefato(df, titulo):
    #Salvar o DataFrame como arquivo parquet
    arquivo_path = f"{titulo}.parquet"
    df.to_parquet(arquivo_path, index=False)
    # Registrando o DataFrame resultante como um artefato no MLflow
    mlflow.log_artifact(arquivo_path, f"{titulo}.parquet")

In [21]:
#variavel com caminho do csv temporario, este foi criado para ser salvo como um aterfato no mlflow
csv_path = ''

def dadosNulosArtefato(df,titulo):
    # Contagem de valores nulos por coluna
    valores_nulos_por_coluna = df.isnull().sum()

    # Criando um DataFrame para exibir os resultados
    tabela_valores_nulos = pd.DataFrame(valores_nulos_por_coluna, columns=['Valores Nulos'])

    # Resetando o índice para manter os nomes das colunas
    tabela_valores_nulos.reset_index(inplace=True)

    # Salvando o DataFrame como um arquivo CSV temporário
    csv_path = f"{titulo}.csv"
    tabela_valores_nulos.to_csv(csv_path, index=False)
    
    #Adicionado um atefato em formato csv com para listar os campos nullos
    mlflow.log_artifact(csv_path, artifact_path= f"{titulo}.csv")
    
    os.remove(csv_path) 

In [22]:

csv_shape = ""

def shapeArtefato(df, titulo):
    # Obtendo o número de linhas e colunas do DataFrame
    num_linhas = df.shape[0]
    num_colunas = df.shape[1]
    # Criando um DataFrame com as informações
    info_shape = pd.DataFrame({'Número de Linhas': [num_linhas], 'Número de Colunas': [num_colunas]})

    # Salvando o DataFrame como um arquivo CSV temporário
    csv_shape = f"{titulo}.csv"
    info_shape.to_csv(csv_shape, index=False)
    #Adicionado artefato com a quantidade inicial de linha e colunas 
    mlflow.log_artifact(csv_shape, artifact_path=f"{titulo}.csv")
    os.remove(csv_shape)

In [23]:
#Criando artefato com os nomes da colunas originais e sua tipagem
def colunasArtefato(df, titulo):
    # Capturando as informações de df.info() em uma string
    info_buffer = StringIO()
    df.info(buf=info_buffer)
    info_text = info_buffer.getvalue()
    mlflow.log_text(info_text, artifact_file=f"{titulo}.txt")

In [24]:
def dataFrameArtefato(df, titulo):
    #Colunas que irão permanecer na base de dados
    colunas_selecionadas = ['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_made_flag']
    #excutando a remoção de colunas e linha que tenha valores nuloes    
    df_processado = df[colunas_selecionadas].dropna()
    colunasArtefato(df_processado, f"Colunas{titulo}")
    shapeArtefato(df_processado, f"shape{titulo}")
    dadosNulosArtefato(df_processado,f"valoresNulos{titulo}") 
    
    # Registrando o DataFrame resultante como um artefato no MLflow
    adicionaDataFrameArtefato(df_processado,titulo)
    
 

In [25]:

#MLflow
with mlflow.start_run(experiment_id=experiment_id, run_name='PreparaçãoDados'):
  # dataFrameArtefato(df,"dataset_kobe_dev") #Adiciondo o data frame de desenvolvimento com atefato no mlflow
  # dataFrameArtefato(df_prod,"dataset_kobe_prod") #Adiciondo o data frame de  produção com atefato no mlflow
   
   dadosNulosArtefato(df, "ValoresNulos") #Adicionado um atefato em formato csv com para listar os campos nullos
   shapeArtefato(df,"ShapeInicial")#Adicionado artefato com a quatidade de linhas e colunas iniciais
   colunasArtefato(df,"colunasIniciais")#Adicionado artefato com os nomes das colunas e sua respecitva tipagem
   dataFrameArtefato(df,"Processado") #Processando o data frame removendo algumas colunas e salvando o mesmo no mlflow
   






# Removendo o arquivo CSV temporário caso exista
